Streamlit (Deployment)

In [ ]:
!pip install streamlit

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd

# Load dataset
df = pd.read_csv('/content/streamlit.csv')

# Ubah kolom target menjadi biner
df['Kelayakan'] = df['Kelayakan'].map({'Tidak Layak': 0, 'Layak': 1})

# Gabungkan semua kolom fitur menjadi satu kolom teks
df['combined_text'] = df.iloc[:, :-1].astype(str).agg(' '.join, axis=1)

# Pisahkan fitur dan target
X = df['combined_text']  # Gabungan teks fitur
y = df['Kelayakan']      # Kolom target

# Vectorize data menggunakan TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)  # Hasil berupa matriks sparse

# Simpan vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=0)

# Inisialisasi dan fit model SVC
classifier = DecisionTreeClassifier(random_state=42)
# classifier = SVC(kernel='linear')
# classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Prediksi
y_pred = classifier.predict(X_test)

# Evaluasi akurasi
score = accuracy_score(y_test, y_pred)
print(f'Akurasi: {score:.4f}')

In [ ]:
import pickle

pickle_out = open("classifier.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [ ]:
%%writefile app.py
import pandas as pd
import pickle
import streamlit as st

# Load model dan vectorizer
pickle_in = open('classifier.pkl', 'rb')
classifier = pickle.load(pickle_in)

pickle_vectorizer = open('tfidf_vectorizer.pkl', 'rb')
vectorizer = pickle.load(pickle_vectorizer)

# Load data
@st.cache_data
def load_data():
    df = pd.read_csv('/content/streamlit.csv')  # Ganti dengan file Anda
    return df

df = load_data()

# Fungsi prediksi
def prediction(title, company, location, salary):
    # Gabungkan input dalam satu string atau array
    user_input = f"{title} {company} {location} {salary}"

    # Transform input menggunakan TF-IDF
    input_vectorized = vectorizer.transform([user_input])

    # Prediksi menggunakan model
    prediction = classifier.predict(input_vectorized)

    # Kembalikan hasil prediksi
    job_map = {0: 'Tidak Layak', 1: 'Layak'}
    job_name = job_map[prediction[0]]
    return job_name

# Main app
def main():
    st.title('Prediksi Kelayakan Kandidat Kerja')

    # Membuat dropdown dengan variasi unik dari data
    title_options = df['Title'].unique()
    company_options = df['Company'].unique()
    location_options = df['Location'].unique()
    salary_options = df['Salary'].unique()

    # Dropdown untuk input
    title = st.selectbox('Pilih Title', title_options)
    company = st.selectbox('Pilih Company', company_options)
    location = st.selectbox('Pilih Location', location_options)
    salary = st.selectbox('Pilih Salary', salary_options)

    result = ""

    if st.button('Prediksi'):
        result = prediction(title, company, location, salary)
    st.success(result)

if __name__ == '__main__':
    main()

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501